In [ ]:
import serial
import time
import codecs

def fixlat(data):
    lat_int=float(data[0:2])
    lat_float=float(data[2:])/60
    return lat_int+lat_float

def fixlng(data):
    lng_int=float(data[0:3])
    lng_float=float(data[3:])/60
    return lng_int+lng_float

def initialize():
# initialise serial port settings
    Port = serial.Serial()
    Port.baudrate = 9600#115200
    Port.port = '/dev/ttyUSB0'

    Port.xonxoff = 1
# return the port as an object we can use
    return Port

def getdata(port):
# open the serial port
    port.open()
# check that the port is open
    if port.isOpen():
# read 16 line
       while True:
        line=str(port.readline())
        if line.find('RMC')>0:
            if line.find('A')>0:
                lat=line[line.index('A')+1:line.index('N',10)].strip(',')
                lng=line[line.index('N',10)+1:line.index('E')].strip(',')
                flng = fixlng(lng)
                flat = fixlat(lat)
               	break
# close the serial port
    port.close()
# discard the first line (sometimes it contains rubbish, so just always discard it)
    #del line[0]
# return the list of lines
    return flng, flat

ser=serial.Serial('/dev/ttyXRUSB1', 9600, timeout = 0.5)

flag = 20
while True:
    ser.write(bytes("AT+NSOCR=DGRAM,17,8888,1\r\n", 'UTF-8'))
    response = ''
    for _ in range(5):
        read =  ser.readline().decode("utf-8")
        if read!='\r\n' and read!='\n' and read!='\r':
            response += read
    for i in range(len(response)):
        print(i, response[i])
    if len(response) > 4:
        if response[3] == 'O' and response[0] != 'E' :
            socketID = int(response[0])
            break
    elif flag == 0:
        print('Fail Connection')
        sys.exit(0)
    else:
        flag -= 1
        continue
    
ser.write(bytes("AT+NPING=120.126.136.17\r\n", 'UTF-8'))
SendtoServer = 'AT+NSOST=' + str(socketID) + ',120.126.136.17,5687'
count = 0
while True:
    count += 1  
    message = str(time.time())
    messageHexAscii = ''
    sPort = initialize()#GPS initialize
    lng, lat = getdata(sPort)#GPS data
    message = str(round(lng,5))+ ':' +str(round(lat,5)) + ':' + str(count) + ':' + message  
    messageByteNumber = str(len(message.encode('utf-8')))
    for x in message:
        messageHexAscii += format(ord(x), "x")
    if ser.is_open:
        ser.write(bytes(SendtoServer + ',' + messageByteNumber + ','
                + messageHexAscii + '\r\n', 'UTF-8'))
    time.sleep(10)

ser.close()
    